In [39]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

In [40]:
load_dotenv()
llm = ChatOpenAI()

In [41]:
class jokeState(TypedDict):
    topic: str 
    joke: str 
    explanation: str 

In [42]:
def generate_joke(state: jokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [43]:
def generate_explanation(state: jokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [44]:
state = StateGraph(jokeState)

state.add_node('generate_joke', generate_joke)
state.add_node('generate_explanation', generate_explanation)

state.add_edge(START, 'generate_joke')
state.add_edge('generate_joke', 'generate_explanation')
state.add_edge('generate_explanation', END)

checkpointer = MemorySaver()

workflow = state.compile(checkpointer=checkpointer)

config1 = {"configurable": {"thread_id": "1"}}

In [47]:
initial_state = {
    'topic': "Create a joke about monkey"
}

workflow.invoke(initial_state, config= config1)

{'topic': 'Create a joke about monkey',
 'joke': 'Why did the monkey like the banana wearing sunglasses?\n\nBecause it was so a-peel-ing!',
 'explanation': 'This joke plays on the pun between "appealing" (attractive or interesting) and "a-peel-ing" (referring to the peel of a banana). The monkey liked the banana wearing sunglasses because it was "a-peel-ing" - both literally in terms of the banana\'s peel, and figuratively because the sunglasses made the banana look cool or attractive.'}